## 5.3.- Servidor WEB

En esta seccion se describira como implemetar un servidor web en el SOC ESP8266 que permita, en este caso prender o apagar un led. Se utilizara una libreria especialmente desarrollada con este fin.
### 5.3.1 Librería ESP8266WebServer
 
Un servidor web es una computadora que almacena los archivos que componen un sitio web (ej.  documentos HTML , imágenes, hojas de estilos CSS y archivo JavaScript) y los entrega a otro dispositivo que le solicita un servicio. Se desarrollo para el ESP8266 una libreria que permite implementar un servidor web. Se denomina libreria esp8266WebServer. Los metodos que se utilizan se muestran a continuacion

| Metodo          | Sintaxis                   |  Funcion      |
|---              |---                         |---            |
|ESP8266WebServer |ESP8266WebServer server(80) | Crea el objeto server |
|begin            | server.begin()             | Inicializa el servidor|
|close            | server.close()             | Apaga el servidor     |
|stop             | server.stop()              | Apaga el servidor     |
| on              | server.on("/", FuncionM);  | Indica una función que emitira una respuesta del servidor cuando un navegador solicita la URL |
|addHandler       | server. addHandler();      |              |
| onNotFound      | server.onNotFound();       |              |
| onFileUpload    | server.onFileUpload();     |              |
| send            | server.send(code,tipo_contenido,contenido)| Envia las respuestas solicitadas code:200, 404 tipo_contenido:"text/plain","image/png"|
|streamFile      |server.streamFile(file, GetContentType(path));| Permite cargar una pagina completa |
|handleClient     |server.handleClient();      |  Maneja las conexiones entrantes|


### 5.3.2 PROGMEM

Permite cargar informacion en la memoria flash. Despues se puede pasar a la RAM cuando hace falta. Es una característica Arduino AVR que ha sido portada a ESP8266 para asegurar la compatibilidad con las librerías existentes en Arduino, así como para ahorrar RAM. La sintaxis seria:

``` const char MAIN_page[] PROGMEM = R"=====( pagina web )======" ```


### 5.3.3 Programa Webserver
Este programa debe prender o apagar el led que se encuentra en la placa desde una pagina WEB. Este sistema puede ser controlado desde un telefono celular. Se utiliza el microcontrolador esp8266 que se conecta a la red WIFI y crea un webserver. Utliza la memoria RAM para guardar la pagina WEB.

Se crea un nuevo programa en el IDE de arduino denominado internet1.ino y los archivos config.h y wifi.hpp se copian en el nuevo directorio.


#### Archivo config.h 

En el archivo config.h se escribe la pagina WEB que se muestra mas abajo

   
<!DOCTYPE html>  
<html>  
<body>  
<center>
<h1> CURSO DE ROBOTICA EDUCATIVA</h1>
<h1>Programa para prender/apagar un LED </h1><br> <br>
Apretar para <a href="ledOn" >Prender LED </a><br>
Apretar para <a href="ledOff">Apagar LED</a><br>
<hr>
</center>

</body>
</html>   
El archivo config.h queda asi:      

```
const char* red_WIFI     = "nombre_red";
const char* clave = "clave";

const char MAIN_page[] PROGMEM = R"=====(  
<!DOCTYPE html>  
<html>  
<body>  
<center>
<h1> CURSO DE ROBOTICA EDUCATIVA</h1>
<h1>Programa para prender/apagar un LED </h1><br> <br>
Apretar para <a href="ledOn" ><button>Prender </button>LED</a><br>
Apretar para <a href="ledOff"><button>Apagar LED</button></a><br>
<hr>
</center>

</body>
</html>
)=====";
```

#### Archivo wifi.hpp

En esp8266 para configurar y controlar un webserver se usan las siguientes funciones.   
   
|  Funcion        |   Sintaxis                   |Acciones                   |   
|---              |---                           |---                        |   
|ESP8266WebServer |  ESP8266WebServer server(80) |Crea el objeto server. |     
| begin           |  server.begin()              |Inicia el servidor web, especificando el puerto (generalmente el 80). |   
| on              |  server.on(/ledOn,funcion    |Permiten definir rutas (URI) y las funciones que se ejecutarán cuando se acceda a esas rutas. |   
|handleClient     |  server.handleClient()       | Escucha las conexiones entrantes y gestiona las solicitudes. |   
|send             |server.send( 200,s)           |Se utilizan para enviar respuestas al cliente, incluyendo código HTML en el string s y OK con 200.  |   
En el archivo wifi.hpp se agregan tienen que estar las siguientes funciones:
- iniciar_server()
- controlRaiz()
- prender()
- apagar()
  

 ##### Funcion iniciar_server   
 Esta funcion es la encargada de enviar la informacion a la pagina web cuando es solicitada, asocia la ruta de la URL raíz ("/") con una función llamada controlRaiz. Cuando un cliente (por ejemplo, un navegador web) solicita la página principal del servidor (la dirección IP del ESP8266), la función controlRaiz se ejecutará para manejar esa solicitud

```
void iniciarServer() {

  server.on("/", controlRaiz);
  server.on("/Prender", prender);  //Rutina mueve el brazo
  server.on("/Apagar", apagar);    //Rutina mueve el brazo
  server.begin();
}

```


##### Funcion controlRaiz 

En este programa carga la pagina web que se encuentra en la memoria RAM y esta definida en el archivo config.h. Se muestra el codigo de la funcion.   
```
void controlRaiz() {  
 Serial.println("Pagina original ");  
 String s = MAIN_page; //Read HTML contents  
 server.send(200, "text/html", s); //Send web page  
}
```

##### Funcion prender 
Esta funcion prende el LED que se encuentra dentro de la placa esp8266.
```  
void prender() {   
 Serial.println("LED prendido");  
 digitalWrite(BUILTIN_LED,LOW); //LED is connected in reverse  
 server.send(200, "text/html", "El LED esta prendido");  
}
```

#####  Funcion apagar 
Funcion apaga el LED que se encuentra dentro de la placa esp8266.
```
void apagar() {   
 Serial.println("LED apagado");  
 digitalWrite(BUILTIN_LED,HIGH); //LED off  
 server.send(200, "text/html", "El LED esta apagado");   
}  
```